## VGG-16 Model

In [2]:
# load the vgg16 model
from keras.applications.vgg16 import VGG16

model = VGG16()
model.summary()


Instructions for updating:
Colocations handled automatically by placer.
553476096/553467096 [==============================] - 98s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__

## Inception-V3

In [4]:
from keras.applications.inception_v3 import InceptionV3

model = InceptionV3()
model.summary()

96124928/96112376 [==============================] - 8s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
____________________________________________

## ResNet50

In [7]:
from keras.applications.resnet50 import ResNet50

model = ResNet50()
model.summary()

102866944/102853048 [==============================] - 21s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
_________________________________________

# Examples of Pre-Trained Models

## VGG-16as classifier

In [25]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import decode_predictions

# load image with the required size
image = load_img('dog.jpg', target_size=(224,224))

# convert image to array
image = img_to_array(image)

# reshape the image 
image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])

# Here the image will pre proccessed before feeding to the model
image = preprocess_input(image)

# load the model
model = VGG16()

# predict the probability of where the input belong to across all the classes of the model.
yhat = model.predict(image)

# convert the probability to class label
label = decode_predictions(yhat)
print(label)

# retireve the most likely result (highest probability)
label = label[0][0]
print(label)

# extarcting the label and the probability
print("check", label[1], label[2]*100)


[[(u'n02107142', u'Doberman', 0.33594823), (u'n02105412', u'kelpie', 0.21615963), (u'n02106550', u'Rottweiler', 0.17698687), (u'n02089078', u'black-and-tan_coonhound', 0.12776816), (u'n02107312', u'miniature_pinscher', 0.037303597)]]
(u'n02107142', u'Doberman', 0.33594823)
('check', u'Doberman', 33.59482288360596)


## VGG-16 as Feature Extractor Pre Proccessor

In [49]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Model
from pickle import dump

image = load_img('dog.jpg', target_size=(224,224))
image = img_to_array(image)
image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])

image = preprocess_input(image)

model = VGG16()

# remove the output layer
model.layers.pop()

model = Model(inputs=model.inputs, outputs=model.layers[-1].output)

# extract the feature
feature = model.predict(image)

print(feature.shape)

# save to file
dump(feature, open('dog.pkl', 'wb'))

(1, 4096)


## VGG-16 as Feature Extractor in Model

In [58]:
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import Model

# Download the vgg-16 model
# setting include_top to False will remove the classifier part from the model
model = VGG16(include_top=False, input_shape=(300,300,3))

# output of the vgg-16 model is given as the input of the flatten layer
flat1 = Flatten()(model.outputs)

# Create a dense layer with activation relu
class1 = Dense(1024, activation='relu')(flat1)

# Create dense layer with 10 class of activaation softmax
output = Dense(10, activation = 'softmax')(class1)

# Create a new model with input as model input and output as the output of vgg-16
# after feeding the inpu and output we can train the new model
# Here weigth of vgg-16 and weight of the new model will be trained together.
model = Model(inputs=model.inputs, outputs=output)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0         
__________

In [59]:
# we can also have the option where we can only train the new layers without updating the weigth of vgg-16

model = VGG16(include_top=False, input_shape=(300,300,3))

# we can make the layers of vgg-16 not trainable so that the weigths will not be updated.
# Here all the layers will be made not trainable
for layer in model.layers:
    layer.trainable = False
    
# to make only certain layers trainable we can select those layers
model.get_layer('block1_conv1').trainable=False
model.get_layer('block1_conv2').trainable=False